<a href="https://colab.research.google.com/github/xc308/Large_Language_Model/blob/main/3_Loading_Models_and_Inference_with_Hugging_Face_Inferences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install transformers # for accessing pretrained models and performing various NLP tasks with ease

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 794.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
from transformers import pipeline
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# suppress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

#Text classification with DistilBERT


##Load the model and tokenizer
initialize a tokenizer and a model for sentiment analysis using DistilBERT fine-tuned on the SST-2 dataset

In [3]:
# Load the tokenizer and model

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

##Preprocess the input text
tokenize the text

In [4]:
text = "Congratulations! You've won a free ticket to the Bahamas. Reply WIN to claim."

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt")

print(inputs)

{'input_ids': tensor([[  101, 23156,   999,  2017,  1005,  2310,  2180,  1037,  2489,  7281,
          2000,  1996, 17094,  1012,  7514,  2663,  2000,  4366,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


##Perform inference

In [6]:
# Perform inference
with torch.no_grad(): # not training the model, so no need to compute and save grad, to save memory
    outputs = model(**inputs) # ** unpack the inputs

##Get the logits
logits are raw, unnormalized predictions of the model

In [8]:
logits = outputs.logits
logits.shape

print(logits)

tensor([[-3.9954,  4.3336]])


##Post-process the output
Convert the logits to probabilities and get the predicted class:

In [9]:
# Convert logits to probabilities
probs = torch.softmax(logits, dim=-1)

# Get the predicted class
predicted_class = torch.argmax(probs, dim=-1)

# Map the predicted class to the label
labels = ["NEGATIVE", "POSITIVE"]
predicted_label = labels[predicted_class]


print(f"Predicted label: {predicted_label}")


Predicted label: POSITIVE


#Text generation with GPT-2

##Load tokenizer

In [10]:
# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")


model = GPT2LMHeadModel.from_pretrained("gpt2")



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

#Preprocess the input text

In [11]:
# Prompt
prompt = "Once upon a time"

# Tokenize the input text
inputs = tokenizer(prompt, return_tensors="pt")
inputs

{'input_ids': tensor([[7454, 2402,  257,  640]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

#Perform inference

In [12]:
# Generate text
output_ids = model.generate(
    inputs.input_ids,  #Input token IDs from the tokenizer
    attention_mask=inputs.attention_mask,  # Mask indicating which tokens to attend to
    pad_token_id=tokenizer.eos_token_id,   # Padding token ID set to the end-of-sequence token ID
    max_length=50,                         # Maximum length of the generated sequences
    num_return_sequences=1                 # Number of sequences to generate
)

output_ids

tensor([[7454, 2402,  257,  640,   11,  262,  995,  373,  257, 1295,  286, 1049,
         8737,  290, 1049, 3514,   13,  383,  995,  373,  257, 1295,  286, 1049,
         3514,   11,  290,  262,  995,  373,  257, 1295,  286, 1049, 3514,   13,
          383,  995,  373,  257, 1295,  286, 1049, 3514,   11,  290,  262,  995,
          373,  257]])

# Post-process the output

Decode the generated tokens to get the text:

In [13]:
# Decode the generated text
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print(generated_text)

Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a


#Hugging Face pipeline() function

- pipeline() function from the Hugging Face transformers library
- It abstracts the complexities of model loading, tokenization, inference, and post-processing, allowing users to perform complex NLP tasks with just a few lines of code.

In [ ]:
## Definition

transformers.pipeline(
    task: str,                     # The task to perform, such as "text-classification", "text-generation", "question-answering"
    model: Optional = None,        # an be a string (model identifier from Hugging Face model hub), a path to a directory containing model files, or a pre-loaded model instance
    config: Optional = None,       # The configuration to use. This can be a string, a path to a directory, or a pre-loaded config object.
    tokenizer: Optional = None,    # The tokenizer to use. This can be a string, a path to a directory, or a pre-loaded tokenizer instance.
    feature_extractor: Optional = None,  # he feature extractor to use for tasks that require it (e.g., image processing).
    framework: Optional = None,          # The framework to use, either "pt" for PyTorch
    revision: str = 'main',              # The specific model version to use
    use_fast: bool = True,               # Whether to use the fast version of the tokenizer if available
    model_kwargs: Dict[str, Any] = None, # Additional keyword arguments passed to the model during initialization.
    **kwargs                             # Additional keyword arguments passed to the pipeline components
)

## Text classification using pipeline()

load a pretrained text classification model and use it to classify a sample text.

In [14]:
# Load a general text classification model
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

# Classify a sample text
result = classifier("Congratulations! You've won a free ticket to the Bahamas. Reply WIN to claim.")
print(result) #The confidence score for the prediction.

Device set to use cpu


[{'label': 'POSITIVE', 'score': 0.9997586607933044}]


##Language detection using pipeline()

load a pretrained language detection model and use it to identify the language of a sample text.

In [15]:
from transformers import pipeline

classifier = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")
result = classifier("Bonjour, comment ça va?")
print(result)

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


[{'label': 'fr', 'score': 0.9934879541397095}]


##Text generation using pipeline()

load a pretrained text generation model and use it to generate text based on a given prompt.

In [16]:
# Initialize the text generation pipeline with GPT-2
generator = pipeline("text-generation", model="gpt2")

Device set to use cpu


In [17]:
# Generate text based on a given prompt
prompt = "Once upon a time"
result = generator(prompt, max_length=50, num_return_sequences=1, truncation=True)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [18]:
result

[{'generated_text': 'Once upon a time in the past few decades, many Americans have simply moved onto a new "political party." Republicans are now in charge — and, in some cases, they won\'t be coming back for another four more years. Indeed, Republicans have'}]

In [19]:
# Print the generated text
print(result[0]['generated_text'])

Once upon a time in the past few decades, many Americans have simply moved onto a new "political party." Republicans are now in charge — and, in some cases, they won't be coming back for another four more years. Indeed, Republicans have


##Text generation using T5 with pipeline()

load a pretrained T5 model and use it to translate a sentence from English to French based on a given prompt.

In [20]:
# Initialize the text generation pipeline with T5
generator = pipeline("text2text-generation", model="t5-small")

# Generate text based on a given prompt
prompt = "translate English to French: How are you?"
result = generator(prompt, max_length=50, num_return_sequences=1)

# Print the generated text
print(result[0]['generated_text'])

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


Comment êtes-vous?


## When to use pipeline()

- Quick Prototyping: need to quickly prototype an NLP application or experiment with different models.

- Simple Tasks: When performing simple or common NLP tasks that are well-supported by the pipeline() function.


## When to avoid pipeline()
- perform highly customized tasks
- need fine-grained control over the model and tokenization process for performance optimization or specific use cases.

## Fill-mask task using BERT with pipeline()

load a pretrained BERT model and use it to predict the masked word in a given sentence.

In [21]:
# Initialize the fill-mask pipeline with BERT
fill_mask = pipeline("fill-mask", model="bert-base-uncased")


# Generate text by filling in the masked token
prompt = "The capital of France is [MASK]."
result = fill_mask(prompt)

# Print the generated text
print(result)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


[{'score': 0.4167894423007965, 'token': 3000, 'token_str': 'paris', 'sequence': 'the capital of france is paris.'}, {'score': 0.07141634821891785, 'token': 22479, 'token_str': 'lille', 'sequence': 'the capital of france is lille.'}, {'score': 0.06339266151189804, 'token': 10241, 'token_str': 'lyon', 'sequence': 'the capital of france is lyon.'}, {'score': 0.04444744810461998, 'token': 16766, 'token_str': 'marseille', 'sequence': 'the capital of france is marseille.'}, {'score': 0.030297260731458664, 'token': 7562, 'token_str': 'tours', 'sequence': 'the capital of france is tours.'}]
